In [ ]:
import os
import numpy as np
import pandas as pd
from sklearn.cluster import KMeans
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt
import seaborn as sns

# ----- 設定 -----
embedding_path = "../data/embeddings/bge_embeddings.npy"  # 埋め込みベクトル
metadata_path = "../data/processed/cleaned_risk_dataset.csv"  # メタ情報（docID, company_name など）
output_plot_path = "../outputs/tsne_clusters.png"

In [ ]:
# ----- データ読み込み -----
print("Loading data...")
embeddings = np.load(embedding_path)
df = pd.read_csv(metadata_path)

# docIDが一致しているか確認（埋め込み数 = データ数）
assert embeddings.shape[0] == len(df), "データ数とベクトル数が一致しません"

In [ ]:
# ----- クラスタリング -----
print("Clustering...")
n_clusters = 9  # 適宜変更
kmeans = KMeans(n_clusters=n_clusters, random_state=42)
df["cluster"] = kmeans.fit_predict(embeddings)

In [ ]:
# ----- 次元圧縮（t-SNE） -----
print("Running t-SNE...")
tsne = TSNE(n_components=2, random_state=42, perplexity=30)
reduced = tsne.fit_transform(embeddings)
df["x"] = reduced[:, 0]
df["y"] = reduced[:, 1]

plt.title("t-SNE plot of Risk Embeddings (by Cluster)")
# 事前にoutputsディレクトリを作成しておく
os.makedirs("outputs", exist_ok=True)
plt.savefig(output_plot_path)
plt.show()
print(f"✅ クラスタプロットを保存しました: {output_plot_path}")

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.manifold import TSNE
from sklearn.cluster import KMeans

# Notebook内描画設定
%matplotlib inline

# 埋め込み読み込み
embeddings = np.load("../data/embeddings/bge_embeddings.npy")

# t-SNEで次元削減
tsne = TSNE(n_components=2, random_state=42, perplexity=30)
embeddings_2d = tsne.fit_transform(embeddings)

# KMeansクラスタリング
kmeans = KMeans(n_clusters=10, random_state=42)
labels = kmeans.fit_predict(embeddings)

# 描画
plt.figure(figsize=(10, 8))
plt.scatter(embeddings_2d[:, 0], embeddings_2d[:, 1], c=labels, cmap="tab10", alpha=0.7)
plt.title("t-SNE Visualization of Risk Embeddings")
plt.xlabel("t-SNE 1")
plt.ylabel("t-SNE 2")
plt.colorbar(label="Cluster")
plt.grid(True)
plt.show()

In [ ]:
# 業種情報を追加
import pandas as pd
df2 = pd.read_csv('../data/processed/nikkei225_complete.csv')
df_complete = pd.merge(df.copy(), df2[['ＥＤＩＮＥＴコード', '提出者業種']], left_on='edinet_code', right_on='ＥＤＩＮＥＴコード', how='left')
df_complete = df_complete.rename(columns={'提出者業種': 'industry'})
df_complete.to_csv('../data/processed/df_rag.csv')

## RAG
構成（llama-index + FAISS + LLM）

In [ ]:
import pandas as pd
df = pd.read_csv('../data/processed/df_rag.csv')

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np

# 1. 全文を使ってTF-IDFを計算
all_texts = df["token_str"].dropna().tolist()
vectorizer_all = TfidfVectorizer(max_features=3000)
tfidf_all = vectorizer_all.fit_transform(all_texts)
global_vocab = vectorizer_all.get_feature_names_out()

# 2. 単語ごとのスコア平均を算出
mean_scores_all = tfidf_all.mean(axis=0).A1

# 3. 上位 N個の「共通語」を取得（除外候補）
top_common = list(set(global_vocab[mean_scores_all.argsort()[::-1][:50]]))

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

stop_words = top_common
vectorizer = TfidfVectorizer(max_features=1000, stop_words=stop_words)

for cluster_id in sorted(df["cluster"].unique()):
    texts = df[df["cluster"] == cluster_id]["token_str"].dropna().tolist()
    
    if not texts:
        continue

    tfidf_matrix = vectorizer.fit_transform(texts)
    feature_names = vectorizer.get_feature_names_out()
    mean_scores = tfidf_matrix.mean(axis=0).A1
    top_n = 10
    top_idx = mean_scores.argsort()[::-1][:top_n]

    print(f"\nクラスタ {cluster_id} の特徴語 Top {top_n}:")
    for i in top_idx:
        print(f"  {feature_names[i]}: {mean_scores[i]:.3f}")

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import japanize_matplotlib  # すでに入ってる場合はスキップ

# 1. 全体の業種比率（期待値）
industry_total_ratio = df_complete["industry"].value_counts(normalize=True)

# 2. クラスタ × 業種 の出現頻度
cluster_industry = df_complete.groupby(["cluster", "industry"]).size().unstack(fill_value=0)

# 3. 各クラスタ内の業種比率
cluster_industry_ratio = cluster_industry.div(cluster_industry.sum(axis=1), axis=0)

# 4. 補正比率（観測 / 期待）
corrected_ratio = cluster_industry_ratio.copy()
for col in corrected_ratio.columns:
    corrected_ratio[col] = corrected_ratio[col] / industry_total_ratio[col]

# 5. ヒートマップ表示（1.0が平均、濃いほど過剰）
plt.figure(figsize=(14, 6))
sns.heatmap(corrected_ratio, annot=True, cmap="RdBu_r", center=1.0, fmt=".2f")
plt.title("クラスタごとの業種構成比（全体比に対する相対値）")
plt.xlabel("業種")
plt.ylabel("クラスタ")
plt.xticks(rotation=45, ha='right')
plt.yticks(rotation=0)
plt.tight_layout()
plt.show()

In [ ]:
df.columns

In [ ]:
# =========================
# ✅ 1. ライブラリ読み込み
# =========================
import pandas as pd
import json
import pickle
import faiss

from llama_index.core import Document, Settings, get_response_synthesizer, VectorStoreIndex
from llama_index.core.query_engine import RetrieverQueryEngine
from llama_index.core.node_parser import SentenceSplitter
from llama_index.core.retrievers import VectorIndexRetriever
from llama_index.vector_stores.faiss import FaissVectorStore
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.core.postprocessor import SentenceTransformerRerank # 追加

from langchain_community.llms.llamacpp import LlamaCpp

# =========================
# ✅ 2. LLMとEmbeddingの設定
# =========================
print("🔧 Setting LLM and embedder...")
llm = LlamaCpp(
    model_path="../models/Llama-3-ELYZA-JP-8B-q4_k_m.gguf",
    temperature=0.0,
    n_ctx=2048,
    max_tokens=512,
    verbose=False
)
Settings.llm = llm

# 埋め込みモデル
Settings.embed_model = HuggingFaceEmbedding(model_name="pkshatech/RoSEtta-base", trust_remote_code=True)

# =========================
# ✅ 3. データ読み込みと準備
# =========================
# Assuming 'df' is loaded and prepared before this section
# For demonstration purposes, let's create a dummy DataFrame if df is not defined

assert "token_str" in df.columns and "cluster" in df.columns, "必須カラムが不足しています"

documents = [Document(text=row["token_str"]) for _, row in df.iterrows()]

# =========================
# ✅ 4. FAISS Indexの作成
# =========================
print("📦 Creating FAISS index...")
dimension = 1024 # Ensure this matches your embedding model's dimension (RoSEtta-base has 1024)
faiss_index = faiss.IndexFlatL2(dimension)
faiss_store = FaissVectorStore(faiss_index=faiss_index)

index = VectorStoreIndex.from_documents(
    documents,
    vector_store=faiss_store,
    transformations=[SentenceSplitter(chunk_size=512, chunk_overlap=20)],
    show_progress=True
)

# オプション：保存
with open("faiss_index.pkl", "wb") as f:
    pickle.dump(index, f)
print("✅ faiss_index.pkl saved.")

In [ ]:
# =========================
# ✅ 5. Query Engine セットアップ (Reranker組み込み)
# =========================
# Retrieverの設定
retriever = VectorIndexRetriever(index=index, similarity_top_k=10) # 最初に多めに取得

# Rerankerの設定
reranker = SentenceTransformerRerank(
    model="hotchpotch/japanese-reranker-cross-encoder-base-v1",
    top_n=3 # リランキング後に最終的に取得するドキュメント数
)

# Response Synthesizerの設定
synthesizer = get_response_synthesizer(response_mode="tree_summarize")

# Query Engineの構築
query_engine = RetrieverQueryEngine(
    retriever=retriever,
    response_synthesizer=synthesizer,
    node_postprocessors=[reranker] # ここでrerankerを追加
)
print("✅ Query Engine with Reranker setup complete.")

In [ ]:
# =========================
# ✅ 6. クラスタごとの要約処理
# =========================
print("🧠 クラスタ要約開始...")
cluster_ids = sorted(df["cluster"].unique())
cluster_summaries = {}

for cluster_id in cluster_ids:
    print(f"\n【クラスタ {cluster_id}】要約中...")

    # 各クラスタに属するドキュメントのみをフィルタリングしてquery_engineに渡す
    # ただし、現状のコードではquery_engineはFAISSインデックス全体に対してクエリを実行します。
    # 特定のクラスタのドキュメントのみを要約したい場合は、
    # そのクラスタのドキュメントを一時的に別のインデックスに入れてクエリするか、
    # 既存のdfからフィルタリングしたデータを使ってSummaryIndexを作成するなどの工夫が必要です。
    # ここでは、簡略化のため、全体のインデックスに対してクエリを発行し、プロンプトで制御することを想定します。

    # クエリにクラスタ固有の情報を追加し、LLMにそのクラスタのドキュメントから要約させる
    # 実際には、cluster_idに対応するドキュメントを抽出して渡す必要がありますが、
    # 現在のLlamaIndexのRetrieverQueryEngineの設計では、
    # クエリ時に動的に特定のクラスタのドキュメントのみを対象とすることは直接的ではありません。
    # そのため、プロンプトで「このクラスタに関連する」というニュアンスを持たせることで、
    # LLMがretrieverから取得したドキュメントの中から関連性の高いものを選択するよう促します。

    # ここをより厳密にするには、cluster_idごとに新しいVectorStoreIndexを作成し、
    # そのindexに対してquery_engineを構築する必要があります。
    # 例:
    # cluster_docs = [Document(text=row["token_str"]) for _, row in df[df["cluster"] == cluster_id].iterrows()]
    # cluster_index = VectorStoreIndex.from_documents(cluster_docs, transformations=[SentenceSplitter(chunk_size=512, chunk_overlap=20)])
    # cluster_retriever = VectorIndexRetriever(index=cluster_index, similarity_top_k=5)
    # cluster_query_engine = RetrieverQueryEngine(retriever=cluster_retriever, response_synthesizer=synthesizer, node_postprocessors=[reranker])
    # response = cluster_query_engine.query(query)

    # 現在のコードの構造を維持しつつ、要約のプロンプトを調整
    query = (
        "以下は同じクラスタに属するリスク文です。\n"
        "このクラスタ特有のリスクの種類・内容を、できるだけ具体的に2～3点、箇条書きで要約してください。\n"
        "抽象的なフレーズ（例：リスク管理、サステナビリティ）ではなく、実際に文中で言及されている内容を用いてください。"
    )

    response = query_engine.query(query)
    summary = str(response).strip()
    cluster_summaries[cluster_id] = summary
    print(f"✅ 要約完了: {summary}")

# =========================
# ✅ 7. JSON形式で保存
# =========================
output_path = "../outputs/reports/cluster_summaries.json"
with open(output_path, "w", encoding="utf-8") as f:
    json.dump(cluster_summaries, f, ensure_ascii=False, indent=2)

print(f"📄 クラスタ要約を {output_path} に保存しました。")

In [ ]:
# 長すぎるクラスタを防ぐために、最大件数を制限（例: 30件）
MAX_DOCS_PER_CLUSTER = 30
cluster_docs_df = cluster_docs_df.head(MAX_DOCS_PER_CLUSTER)

In [ ]:
# =========================
# ✅ 1. ライブラリ読み込み
# =========================
import pandas as pd
import json
import pickle
import faiss

from llama_index.core import Document, Settings, get_response_synthesizer, VectorStoreIndex
from llama_index.core.query_engine import RetrieverQueryEngine
from llama_index.core.node_parser import SentenceSplitter
from llama_index.core.retrievers import VectorIndexRetriever
from llama_index.vector_stores.faiss import FaissVectorStore
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.core.postprocessor import SentenceTransformerRerank

from langchain_community.llms.llamacpp import LlamaCpp

# =========================
# ✅ 2. LLMとEmbeddingの設定
# =========================
print("🔧 Setting LLM and embedder...")
llm = LlamaCpp(
    model_path="../models/Llama-3-ELYZA-JP-8B-q4_k_m.gguf",
    temperature=0.1,
    n_ctx=8192,  # 8192に変更
    max_tokens=512,
    verbose=False,
    n_gpu_layers=32
)
Settings.llm = llm

# 埋め込みモデル
# HuggingFaceEmbeddingのインスタンスを直接変数に格納する
embed_model_instance = HuggingFaceEmbedding(model_name="pkshatech/RoSEtta-base", trust_remote_code=True)
Settings.embed_model = embed_model_instance # Settingsにも設定しておく

# =========================
# ✅ 3. データ読み込みと準備
# =========================
df = pd.read_csv('../data/processed/df_rag.csv')
assert "risk_text" in df.columns and "cluster" in df.columns, "必須カラムが不足しています"

# =========================
# ✅ 4. Rerankerの設定 (一度だけ定義)
# =========================
# Rerankerの設定
reranker = SentenceTransformerRerank(
    model="hotchpotch/japanese-reranker-cross-encoder-base-v1",
    top_n=3 # リランキング後に最終的に取得するドキュメント数
)
print("✅ Reranker setup complete.")


# =========================
# ✅ 5. クラスタごとの要約処理 (メインの変更点)
# =========================
print("🧠 クラスタ要約開始...")
cluster_ids = sorted(df["cluster"].unique())
cluster_summaries = {}

# Node Parser (SentenceSplitter) はここでも使用
node_parser = SentenceSplitter(chunk_size=512, chunk_overlap=20)

for cluster_id in cluster_ids:
    print(f"\n【クラスタ {cluster_id}】要約中...")

    # 1. 特定クラスタのドキュメントのみを抽出
    cluster_docs_df = df[df["cluster"] == cluster_id]
    cluster_documents = [Document(text=row["risk_text"]) for _, row in cluster_docs_df.iterrows()]

    if not cluster_documents:
        print(f"⚠️ クラスタ {cluster_id} にドキュメントがありません。スキップします。")
        cluster_summaries[cluster_id] = "このクラスタには要約するドキュメントがありません。"
        continue

    # 長すぎるクラスタを防ぐために、最大件数を制限（例: 30件）
    MAX_DOCS_PER_CLUSTER = 30
    # ここでdfの件数を制限するのではなく、cluster_documentsのリストを直接制限する
    cluster_documents = cluster_documents[:MAX_DOCS_PER_CLUSTER]


    # 2. そのクラスタのドキュメントから一時的なFAISS Indexを作成
    # 各クラスタでFAISS Indexを再作成するため、次元を再定義
    dimension = 1024 # RoSEtta-baseの次元
    cluster_faiss_index = faiss.IndexFlatL2(dimension)
    cluster_faiss_store = FaissVectorStore(faiss_index=cluster_faiss_index)

    cluster_index = VectorStoreIndex.from_documents(
        cluster_documents,
        vector_store=cluster_faiss_store,
        transformations=[node_parser], # 既存のnode_parserを使用
        embed_model=embed_model_instance, # 明示的に埋め込みモデルを指定する
        show_progress=False # 各クラスタでprogress barを出すと煩雑になるためオフ
    )
    print(f"  --- クラスタ {cluster_id} の一時インデックスを作成しました。")

    # 3. クラスタ専用のQuery Engineを構築
    cluster_retriever = VectorIndexRetriever(index=cluster_index, similarity_top_k=5) # クラスタ内の取得数を5に変更
    synthesizer = get_response_synthesizer(response_mode="tree_summarize") # Synthesizerは再利用可能

    cluster_query_engine = RetrieverQueryEngine(
        retriever=cluster_retriever,
        response_synthesizer=synthesizer,
        node_postprocessors=[reranker] # ここでrerankerを追加
    )
    print(f"  --- クラスタ {cluster_id} のQuery Engineを設定しました。")

    # 4. 要約クエリの実行
    query = (
        "以下は同じクラスタに属するリスク文です。\n"
        "このクラスタ特有のリスクの種類・内容を、できるだけ具体的に2～3点、箇条書きで要約してください。\n"
        "抽象的なフレーズ（例：リスク管理、サステナビリティ）ではなく、実際に文中で言及されている内容を用いてください。\n"
        "出力は日本語で、200文字以内に収めてください。"
    )

    response = cluster_query_engine.query(query)
    summary = str(response).strip()
    cluster_summaries[cluster_id] = summary
    print(f"✅ 要約完了: {summary}")

# =========================
# ✅ 6. JSON形式で保存
# =========================
output_path = "../outputs/reports/cluster_summaries.json"
with open(output_path, "w", encoding="utf-8") as f:
    json.dump(cluster_summaries, f, ensure_ascii=False, indent=2)

print(f"📄 クラスタ要約を {output_path} に保存しました。")

In [ ]:
import pandas as pd
import 
import json
import torch
import gc # ガベージコレクション用

# LLMの接続にopenaiライブラリとLlamaIndexのOpenAILLMラッパーを使用
from openai import OpenAI
from llama_index.llms.openai import OpenAI as LlamaIndexOpenAI

from llama_index.core import Document, Settings, get_response_synthesizer, VectorStoreIndex
from llama_index.core.query_engine import RetrieverQueryEngine
from llama_index.core.node_parser import SentenceSplitter
from llama_index.core.retrievers import VectorIndexRetriever
from llama_index.vector_stores.faiss import FaissVectorStore # faiss-gpu-cu12などをインストールした場合
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.core.postprocessor import SentenceTransformerRerank

# =========================
# 1. LLMとEmbeddingの設定
# =========================
print("🔧 LLMと埋め込みモデルを設定中...")

# LLMの設定 (llama.cpp server + OpenAI互換API経由)
# OpenAIクライアントをllama.cppサーバーに接続します
# api_keyはダミーで問題ありません。
client = OpenAI(
    base_url="http://localhost:8080/v1", # llama-serverがリッスンしているURL
    api_key="dummy_api_key"             # ダミーキーでOK
)

# LlamaIndexのOpenAILLMラッパーを設定
# モデル名もllama-serverではダミーで問題ありません
llm = LlamaIndexOpenAI(
    model="gpt-3.5-turbo",              # 任意のモデル名
    api_key="dummy_api_key",            # ダミーキーでOK
    api_base="http://localhost:8080/v1",# llama-serverがリッスンしているURL
    temperature=0.6,                    # 生成の多様性 (0.0-1.0)。高いほど多様な回答。
    max_tokens=512,                     # LLMが生成する最大トークン数。要約の長さに応じて調整。
    # LlamaIndexのOpenAILLMは、OpenAI APIのチャットテンプレートに準拠するため、
    # llama.cppサーバーが期待するChatML形式と互換性があります。
    # system_promptやquery_wrapper_promptはLlamaIndexが適切に処理します。
)
Settings.llm = llm

# 埋め込みモデルの設定 (これはこれまでと同じ)
# pkshatech/RoSEtta-base は初回実行時に自動でダウンロードされます
embed_model_instance = HuggingFaceEmbedding(model_name="pkshatech/RoSEtta-base", trust_remote_code=True)
Settings.embed_model = embed_model_instance

print("✅ LLMと埋め込みモデルの設定が完了しました。")

# =========================
# 2. データ読み込みと準備
# =========================
# dfは前もって定義されていることを前提とします。
# 例: df = pd.read_csv("your_data.csv")
# あるいは、テスト用にダミーデータを作成（本番ではこれを削除）
df = pd.read_csv('../data/processed/df_rag.csv')

assert "risk_text" in df.columns and "cluster" in df.columns, "DataFrameに必須カラム 'risk_text' または 'cluster' が見つかりません。"

# =========================
# 3. Rerankerの設定
# =========================
# Rerankerの設定 (これはこれまでと同じ)
# hotchpotch/japanese-reranker-cross-encoder-base-v1 も自動ダウンロードされます
reranker = SentenceTransformerRerank(
    model="hotchpotch/japanese-reranker-cross-encoder-base-v1",
    top_n=3 # リランキング後に最終的に取得するドキュメント数
)
print("✅ Rerankerの設定が完了しました。")

# =========================
# 4. クラスタごとの要約処理
# =========================
print("🧠 クラスタごとの要約を開始します...")
# cluster_idsをPythonのint型に変換して取得 (TypeError対策)
cluster_ids = sorted([int(c_id) for c_id in df["cluster"].unique()])
cluster_summaries = {}

# Node Parser (SentenceSplitter)
node_parser = SentenceSplitter(chunk_size=512, chunk_overlap=20)

for cluster_id in cluster_ids:
    print(f"\n--- 【クラスタ {cluster_id}】要約中...")

    # 1. 特定クラスタのドキュメントのみを抽出
    cluster_docs_df = df[df["cluster"] == cluster_id]
    cluster_documents = [Document(text=row["risk_text"]) for _, row in cluster_docs_df.iterrows()]

    if not cluster_documents:
        print(f"⚠️ クラスタ {cluster_id} に要約するドキュメントがありません。スキップします。")
        cluster_summaries[int(cluster_id)] = "このクラスタには要約するドキュメントがありません。"
        continue

    # 長すぎるクラスタを防ぐために、最大件数を制限（例: 30件）
    MAX_DOCS_PER_CLUSTER = 30
    cluster_documents = cluster_documents[:MAX_DOCS_PER_CLUSTER]

    # 2. そのクラスタのドキュメントから一時的なFAISS Indexを作成
    # RoSEtta-baseの埋め込み次元は1024
    dimension = 1024
    cluster_faiss_index = faiss.IndexFlatL2(dimension)
    cluster_faiss_store = FaissVectorStore(faiss_index=cluster_faiss_index)

    cluster_index = VectorStoreIndex.from_documents(
        cluster_documents,
        vector_store=cluster_faiss_store,
        transformations=[node_parser],
        embed_model=embed_model_instance, # 明示的に埋め込みモデルを指定
        show_progress=False
    )
    print(f"  --- クラスタ {cluster_id} の一時インデックスを作成しました。")

    # 3. クラスタ専用のQuery Engineを構築
    cluster_retriever = VectorIndexRetriever(index=cluster_index, similarity_top_k=5)
    synthesizer = get_response_synthesizer(response_mode="tree_summarize")

    cluster_query_engine = RetrieverQueryEngine(
        retriever=cluster_retriever,
        response_synthesizer=synthesizer,
        node_postprocessors=[reranker]
    )
    print(f"  --- クラスタ {cluster_id} のQuery Engineを設定しました。")

    # 4. 要約クエリの実行
    # ここでのクエリはLLMに直接渡されます
    query = (
        "このクラスタに属するリスク文を2～3点、箇条書きで具体的に要約してください。\n"
        "抽象的なフレーズは避け、文中で言及されている内容のみを用いてください。\n"
        "出力は日本語で、200文字以内に収めてください。"
    )

    try:
        response = cluster_query_engine.query(query)
        summary = str(response).strip()
        cluster_summaries[int(cluster_id)] = summary # キーが確実にPythonのint型であることを保証
        print(f"✅ 要約完了: {summary}")
    except Exception as e:
        print(f"❌ クラスタ {cluster_id} の要約中にエラーが発生しました: {e}")
        cluster_summaries[int(cluster_id)] = f"要約中にエラーが発生しました: {e}"

# =========================
# 5. JSON形式で保存
# =========================
output_path = "../outputs/reports/cluster_summaries.json"
# outputs/reports ディレクトリが存在しない場合に作成
import os
os.makedirs(os.path.dirname(output_path), exist_ok=True)

with open(output_path, "w", encoding="utf-8") as f:
    json.dump(cluster_summaries, f, ensure_ascii=False, indent=2)

print(f"📄 クラスタ要約結果を {output_path} に保存しました。")

# =========================
# 6. メモリ解放の試み
# =========================
# llama.cppサーバーのモデルは別プロセスでメモリ管理されるため、Pythonスクリプト側での解放は不要です。
# ただし、埋め込みモデルのインスタンスやLlamaIndexのSettingsは削除しておくと良いでしょう。
if 'embed_model_instance' in locals():
    del embed_model_instance
# if 'Settings' in locals() and hasattr(Settings, 'llm'): # この行と次行を削除またはコメントアウト
#     del Settings.llm                                     # この行を削除またはコメントアウト
# if 'Settings' in locals() and hasattr(Settings, 'embed_model'): # この行と次行を削除またはコメントアウト
#     del Settings.embed_model                             # この行を削除またはコメントアウト

# Pythonのガベージコレクタを強制的に実行
gc.collect()

# PyTorchのCUDAキャッシュをクリア（念のため）
if torch.cuda.is_available():
    torch.cuda.empty_cache()

print("GPUメモリの解放を試みました（該当する場合）。")

In [ ]:
df.columns

In [13]:
import json
import pickle
import faiss
from llama_index.core import Document, Settings, get_response_synthesizer, VectorStoreIndex
from llama_index.core.query_engine import RetrieverQueryEngine
from llama_index.vector_stores.faiss import FaissVectorStore
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.llms.llama_cpp import LlamaCPP
from llama_index.core.retrievers import VectorIndexRetriever
from llama_index.core.postprocessor import SentenceTransformerRerank
from llama_index.core.node_parser import SentenceSplitter # for chunking
from llama_index.core.prompts import PromptTemplate
import pandas as pd
from collections import defaultdict
from kneed import KneeLocator
import numpy as np # for KMeans
import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings("ignore")

# --- LLMと埋め込みモデルの設定 ---
# LLM (llama-server)
Settings.llm = LlamaCPP(
    model_path="../models/Llama-3-ELYZA-JP-8B-q4_k_m.gguf",
    temperature=0.1,
    max_new_tokens=1024, # 要約に必要な最大トークン数 (必要に応じて調整)
    context_window=8192, # Llama 3 ELYZA JP 8B のコンテキストウィンドウに合わせて調整
    # その他、llama-serverで設定したパラメータがあればここに追加できる
    # 例: n_gpu_layers=32 (ただしこれはサーバー側で設定済みなので不要なはず)
    # LlamaIndex 0.10.x 以降では、これらの引数は LlamaCPP 初期化時に渡す
)

# Embedding model
Settings.embed_model = HuggingFaceEmbedding(
    model_name="pkshatech/RoSEtta-base",
    trust_remote_code=True
)

# --- Rerankerの設定 ---
reranker = SentenceTransformerRerank(
    model="hotchpotch/japanese-reranker-cross-encoder-base-v1",
    top_n=3,
    device="cuda"
)

# --- データ読み込み ---
# 仮に 'data/processed/all_risk_sentences.csv' に全ての期間のリスク文があるとする
# ここでは 'cleaned_risk_text' カラムにリスク文、'date' カラムに日付が入っていると仮定
df_all_risks = pd.read_csv('../data/processed/df_rag.csv')

# 'date' カラムから 'year' カラムを抽出
df_all_risks['year'] = pd.to_datetime(df_all_risks['date']).dt.year

# --- 年ごとのループ処理 ---
start_year = 2018
end_year = 2024 # 2024年のデータも対象に含める場合

# outputsディレクトリがなければ作成
import os
os.makedirs('../outputs/reports', exist_ok=True)

for current_year in range(start_year, end_year + 1):
    print(f"\n🧠 【{current_year}年】クラスタごとの要約を開始します...")

    # 指定年のデータのみをフィルタリング
    df_year = df_all_risks[df_all_risks['year'] == current_year].copy()

    if df_year.empty:
        print(f"--- {current_year}年のリスクデータが見つかりませんでした。スキップします。")
        continue

    # フィルタリングされたデータからDocumentオブジェクトを作成
    documents = [Document(text=row['cleaned_risk_text'], metadata={
                    'doc_id': row['doc_id'],
                    'date': row['date'],
                    'company_name': row['filer_name'],
                    'edinet_code': row['edinet_code'],
                    'sec_code': row['sec_code'],
                    'industry': row['industry'],
                 }) for index, row in df_year.iterrows()]

    # --- クラスタリング処理 (年ごとに実行) ---
    from sklearn.cluster import KMeans
    from sklearn.preprocessing import normalize

    print(f"--- 【{current_year}年】埋め込みベクトルの生成中...")
    # LlamaIndexの埋め込みモデルを使って、Documentsから直接埋め込みを生成する
    # これは時間がかかる処理なので、もし既に生成済みの埋め込みがある場合は、それをロードする方が効率的
    embeddings_list = [Settings.embed_model.get_text_embedding(doc.text) for doc in documents]
    
    # numpy 配列に変換
    X = np.array(embeddings_list)
    
    # 正規化 (KMeansでコサイン類似度に近い結果を得るため)
    X_normalized = normalize(X)

    # --- エルボー法による最適なクラスタ数の決定 ---
    # ドキュメント数が少ない場合はクラスタ数を制限する
    # 各クラスタに最低でも2つのドキュメントがあることを想定し、Kの最大値を調整
    max_k_for_elbow = min(len(documents) // 2, 15)
    
    # 最小クラスタ数を2に設定（1だとクラスタリングの意味がないため）
    if max_k_for_elbow < 2:
        print(f"--- {current_year}年のドキュメント数 ({len(documents)}) が少なすぎるため、クラスタリングをスキップします。最低2つのクラスタリングには4つ以上のドキュメントが必要です。")
        continue

    distortions = []
    # Kの範囲を2からmax_k_for_elbowまで試す
    K_range = range(2, max_k_for_elbow + 1)

    print(f"--- 【{current_year}年】最適なクラスタ数を見つけるためエルボー法を実行中 (K=2～{max_k_for_elbow})...")
    for k in K_range:
        kmeans = KMeans(n_clusters=k, random_state=42, n_init='auto')
        kmeans.fit(X_normalized)
        distortions.append(kmeans.inertia_)

    # KneeLocatorを使って最適なKを自動決定
    # S=1.0 はエルボーの鋭さの感度。調整が必要な場合もある
    kneedle = KneeLocator(K_range, distortions, S=1.0, curve="convex", direction="decreasing")
    optimal_n_clusters = kneedle.elbow

    # 最適なKが見つからなかった場合や、K=1（意味がない）になる場合などへの対策
    if optimal_n_clusters is None or optimal_n_clusters < 2:
        # ドキュメント数に応じてデフォルト値を設定
        if len(documents) >= 100: # ドキュメント数が多い場合
            optimal_n_clusters = 5
        elif len(documents) >= 20: # 中程度の場合
            optimal_n_clusters = 3
        else: # 少ない場合
            optimal_n_clusters = 2
        print(f"--- 【{current_year}年】自動検出で最適なクラスタ数が見つからなかったか、小さすぎました。デフォルト値 {optimal_n_clusters} を使用します。")
    
    print(f"--- 【{current_year}年】最適なクラスタ数: {optimal_n_clusters}")

    # エルボープロットの描画と保存（最適点を強調）
    plt.figure(figsize=(10, 6))
    plt.plot(K_range, distortions, marker='o')
    if optimal_n_clusters: # 最適なKが見つかっていればプロットに点を打つ
        plt.vlines(optimal_n_clusters, plt.ylim()[0], plt.ylim()[1], linestyles='dashed', colors='red', label=f'Optimal K = {optimal_n_clusters}')
        plt.legend()
    plt.xlabel('Number of clusters (K)')
    plt.ylabel('Distortion (Inertia)')
    plt.title(f'Elbow Method for Optimal K ({current_year}) - Optimal K: {optimal_n_clusters}')
    plt.grid(True)
    
    elbow_plot_filename = f'../outputs/elbow_plots/elbow_method_{current_year}.png'
    plt.savefig(elbow_plot_filename)
    plt.close()
    print(f"--- 【{current_year}年】エルボープロットを '{elbow_plot_filename}' に保存しました。")

    # --- クラスタリング (最適なn_clustersで実行) ---
    print(f"--- 【{current_year}年】KMeansクラスタリング実行中 (K={optimal_n_clusters})...")
    kmeans = KMeans(n_clusters=optimal_n_clusters, random_state=42, n_init='auto')
    kmeans_labels = kmeans.fit_predict(X_normalized)

    # クラスタリング結果をDocumentオブジェクトに付与
    for i, doc in enumerate(documents):
        doc.metadata['cluster_label'] = int(kmeans_labels[i]) # NumPy int64をPython intに変換

    # クラスタごとにドキュメントをグループ化
    clustered_documents = defaultdict(list)
    for doc in documents:
        clustered_documents[doc.metadata['cluster_label']].append(doc)

    # --- クラスタごとの要約ループ ---
    all_cluster_summaries = {}

    for cluster_label, cluster_docs in sorted(clustered_documents.items()):
        print(f"--- 【クラスタ {cluster_label}】要約中...")

        # クラスタにドキュメントがない場合はスキップ
        if not cluster_docs:
            print(f"--- クラスタ {cluster_label} にドキュメントがありません。スキップします。")
            continue

        # FAISSインデックスの次元を設定（RoSEtta-baseは1024次元）
        dimension = 1024 
        
        # 毎回新しいFaissIndexインスタンスを作成し、そこへドキュメントを埋め込み・追加
        # ここで`faiss.IndexFlatL2`を使っているが、`faiss.IndexFlatIP`（内積）の方がコサイン類似度に近い
        # (ただし、埋め込みベクトルが正規化されている場合はL2でも良い結果になる)
        cluster_faiss_index = faiss.IndexFlatL2(dimension) 
        cluster_faiss_store = FaissVectorStore(faiss_index=cluster_faiss_index)

        cluster_index = VectorStoreIndex.from_documents(
            cluster_docs,
            vector_store=cluster_faiss_store,
            transformations=[SentenceSplitter(chunk_size=4096, chunk_overlap=20)],
            show_progress=False
        )

        # Query Engineの設定
        retriever = VectorIndexRetriever(
            index=cluster_index,
            similarity_top_k=5, # 関連性の高い上位K件を取得
        )
        my_prompt = PromptTemplate(
            template="""
        以下は、事業リスクに関する情報です。
        {context_str}
        
        これらの情報から、リスク文を2～3点、箇条書きで具体的に要約したものを出力してください。
        抽象的なフレーズを避け、文中で言及されている内容のみを用いて、要約したリスク文を箇条書きで具体的に出力してください。
        **要約結果のみを簡潔に箇条書きで出力してください。それ以外の前書きや説明は一切不要です。**
        """
        )
        response_synthesizer = get_response_synthesizer(
            response_mode="compact", # 要約モード
            text_qa_template=my_prompt
        )
        query_engine = RetrieverQueryEngine(
            retriever=retriever,
            response_synthesizer=response_synthesizer,
            node_postprocessors=[reranker],
        )

        # 要約クエリの実行
        query_text = f"このクラスタにまとめられている主要な{current_year}年の事業リスクは何ですか？" # 年をクエリに含める
        response = query_engine.query(query_text)
        
        summary_text = str(response)
        all_cluster_summaries[f"cluster_{cluster_label}"] = {
            "summary": summary_text,
            "document_count": len(cluster_docs),
            "sample_risk_sentences": [doc.text for doc in cluster_docs[:3]] # 代表的なリスク文を3つ含める
        }

        print(f"✅ 要約完了: クラスタ {cluster_label}")
        print(summary_text)

    # --- 結果の保存 (年ごとのファイル名に変更) ---
    output_filename = f'../outputs/reports/cluster_summaries_{current_year}.json'
    with open(output_filename, 'w', encoding='utf-8') as f:
        json.dump(all_cluster_summaries, f, ensure_ascii=False, indent=4)
    print(f"\n--- 【{current_year}年】の要約結果を '{output_filename}' に保存しました。")

print("\n--- 全期間の年ごとのクラスタリングと要約プロセスが完了しました ---")

llama_model_loader: loaded meta data with 22 key-value pairs and 291 tensors from ../models/Llama-3-ELYZA-JP-8B-q4_k_m.gguf (version GGUF V3 (latest))
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.name str              = Llama-3-8B-optimal-merged-stage2
llama_model_loader: - kv   2:                          llama.block_count u32              = 32
llama_model_loader: - kv   3:                       llama.context_length u32              = 8192
llama_model_loader: - kv   4:                     llama.embedding_length u32              = 4096
llama_model_loader: - kv   5:                  llama.feed_forward_length u32              = 14336
llama_model_loader: - kv   6:                 llama.attention.head_count u32              = 32
llama_model_loader: - kv   7:              lla


🧠 【2018年】クラスタごとの要約を開始します...
--- 【2018年】埋め込みベクトルの生成中...
--- 【2018年】最適なクラスタ数を見つけるためエルボー法を実行中 (K=2～15)...
--- 【2018年】最適なクラスタ数: 8
--- 【2018年】エルボープロットを '../outputs/elbow_plots/elbow_method_2018.png' に保存しました。
--- 【2018年】KMeansクラスタリング実行中 (K=8)...
--- 【クラスタ 0】要約中...


Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pai

✅ 要約完了: クラスタ 0
2018年、当社グループにまとめられている主要な事業リスクは以下のとおりです。

1. 消費増税や五輪後の不況発生に係るリスク
2. 少子高齢化長寿命化に係るリスク
3. 所得の二極化に係るリスク
4. テクノロジーの進化に係るリスク
5. ＣＳＲの重要性アップに係るリスク
6. 減損に係るリスク
7. 資金調達に係るリスク
8. 資産運用に関するリスク
9. 国内外の自然災害による多額の保険金支払のリスク
10. 流動性リスク
11. 再保険取引に関するリスク
12. 経済環境社会環境等の予期せぬ変化により損失が発生するリスク
13. 保険業界の競争に関するリスク
14. 国内生命保険事業に関するリスク
15. 海外事業に関するリスク
16. 関連事業に関するリスク
17. 情報漏洩等に関するリスク
18. システムリスク
19. 人事労務に関するリスク
20. 事業運営に関するリスク
21. 事業中断に関するリスク
22. 統合のシナジーが十分に発揮されないリスク
--- 【クラスタ 1】要約中...


Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pai

...

✅ 要約完了: クラスタ 3
2024年の事業リスクとして、以下の点が挙げられます。

1. プリンターの売上変動がエプソンの経営成績に影響を及ぼす可能性がある。
2. 他社との競合が激化し、販売価格の低下や低価格品への需要のシフト、販売数量の減少などが発生する可能性がある。
3. エプソンの技術が競合他社の技術に追い越される、もしくは新しい技術が登場した場合、エプソンの技術的な競争優位性が損なわれ、経営成績に影響を及ぼす可能性がある。

Original Answer: 
- 1. プリンターの売上変動がエプソンの経営成績に影響を及ぼす可能性がある。
         - 2. 他社との競合が激化し、販売価格の低下や低価格品への需要のシフト、販売数量の減少などが発生する可能性がある。
         - 3. エプソンの技術が競合他社の技術に追い越される、もしくは新しい技術が登場した場合、エプソンの技術的な競争優位性が損なわれ、経営成績に影響を及ぼす可能性がある。

Please note that the refined answer is based on the provided context and may not be exhaustive.
--- 【クラスタ 4】要約中...


Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pai

✅ 要約完了: クラスタ 4
 - 与信費用の増加のリスク
         - 保有有価証券の評価損益の悪化のリスク
         - サイバー攻撃やシステム障害等の危機発生のリスク
         - 中長期的な視点から、景気変動や競合他社の参入、業務範囲の拡大に伴うリスク、金利の変動、気候変動等のリスクが存在する。
         - 短期的な視点から、信用リスク、特定の取引先等への高い依存度、地域経済の動向、市場リスク、流動性リスク、退職給付債務に関するリスク等のリスクが存在する。

Note: The refined answer is based on the provided context and the original answer.
--- 【クラスタ 5】要約中...


Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pai

✅ 要約完了: クラスタ 5
2024年の主要な事業リスクは、景気変動や社会的変革に伴うリスク、人材の確保や育成に伴うリスク、構造改革の進捗に伴うリスク、競争環境や構造変化に起因するリスク、地政学リスク、サプライチェーンリスク、情報セキュリティリスク、クライシス危機に伴うリスクなどが挙げられます。

--- 【2024年】の要約結果を '../outputs/reports/cluster_summaries_2024.json' に保存しました。

--- 全期間の年ごとのクラスタリングと要約プロセスが完了しました ---
